[View in Colaboratory](https://colab.research.google.com/github/josd/eye/blob/master/transduction/transduction_dices/observation_prediction_dices.ipynb)

# Transduction from observation to prediction for dices

In [1]:
# Preparation

% rm -fr ~/t2t_data/observation_prediction_dices
% rm -fr ~/t2t_train/observation_prediction_dices/transformer-transformer_small/

In [2]:
# See the observation_prediction_dices problem

! pygmentize -g observation_prediction_dices.py

import random
from tensor2tensor.data_generators import problem
from tensor2tensor.data_generators import text_problems
from tensor2tensor.utils import registry

@registry.register_problem
class ObservationPredictionDices(text_problems.Text2TextProblem):
  """Transduction from observation to prediction for dices."""

  @property
  def approx_vocab_size(self):
    return 256

  @property
  def is_generate_per_split(self):
    # generate_data will shard the data into TRAIN and EVAL for us.
    return False

  @property
  def dataset_splits(self):
    """Splits of data to produce and number of output shards for each."""
    return [{
        "split": problem.DatasetSplit.TRAIN,
        "shards": 7,
    }, {
        "split": problem.DatasetSplit.EVAL,
        "shards": 3,
    }]

  def generate_samples(self, data_dir, tmp_dir, dataset_split):
    del data_dir
    del tmp_dir
    del dataset_split

    for n in range(10000):
      outcome = random.randint(1, 6)
      yield {
        "inputs

In [3]:
# See the observation_prediction_dices script

! pygmentize -g observation_prediction_dices.sh

#!/bin/bash
PROBLEM=observation_prediction_dices
MODEL=transformer
HPARAMS=transformer_small

USER_DIR=$PWD
DATA_DIR=$HOME/t2t_data/$PROBLEM
TMP_DIR=/tmp/t2t_datagen/$PROBLEM
TRAIN_DIR=$HOME/t2t_train/$PROBLEM/$MODEL-$HPARAMS

mkdir -p $DATA_DIR $TMP_DIR $TRAIN_DIR

# Generate data
t2t-datagen \
  --data_dir=$DATA_DIR \
  --problem=$PROBLEM \
  --t2t_usr_dir=$USER_DIR \
  --tmp_dir=$TMP_DIR

# Train
t2t-trainer \
  --data_dir=$DATA_DIR \
  --eval_steps=10 \
  --hparams_set=$HPARAMS \
  --local_eval_frequency=100 \
  --model=$MODEL \
  --output_dir=$TRAIN_DIR \
  --problem=$PROBLEM \
  --t2t_usr_dir=$USER_DIR \
  --train_steps=1000 \
  --worker_gpu_memory_fraction=0.75

# Decode
t2t-decoder \
  --data_dir=$DATA_DIR \
  --decode_from_file=sample_dices.observation \
  --decode_hparams="beam_size=3,alpha=0.6,return_beams=True,write_beam_scores=True" \
  --decode_to_file=sample_dices.prediction \
  --hparams_set=$HPARAMS \
  --model=$MODEL \
  --output_dir=$TRAIN_DIR \
  --problem=$PROBLEM 

In [ ]:
# Run the observation_prediction_dices script

! ./observation_prediction_dices.sh

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
INFO:tensorflow:Importing user module transduction_dices from path /home/jdroo/github.com/josd/eye/transduction
[2018-05-15 00:04:06,412] Importing user module transduction_dices from path /home/jdroo/github.com/josd/eye/transduction
INFO:tensorflow:Generating problems:
    observation:
      * observation_prediction_dices
[2018-05-15 00:04:06,418] Generating problems:
    observation:
      * observation_prediction_dices
INFO:tensorflow:Generating data for observation_prediction_dices.
[2018-05-15 00:04:06,419] Generating data for observation_prediction_dices.
INFO:tensorflow:Generating vocab file: /home/jdroo/t2t_data/observation_prediction_dices/vocab.observation_prediction_dices.256.

Instructions for updating:
When switching to tf.estimator.Estimator, use tf.estimator.RunConfig instead.
[2018-05-15 00:05:02,517] From /usr/local/lib/python3.5/dist-packages/tensor2tensor/utils/trainer_lib.py:151: RunConfig.__init__ (from tensorflow.contrib.learn.python.learn.estimators.run_config) is deprecated and will be removed in a future version.
Instructions for updating:
When switching to tf.estimator.Estimator, use tf.estimator.RunConfig instead.
INFO:tensorflow:schedule=continuous_train_and_eval
[2018-05-15 00:05:02,518] schedule=continuous_train_and_eval
INFO:tensorflow:worker_gpu=1
[2018-05-15 00:05:02,518] worker_gpu=1
INFO:tensorflow:sync=False
[2018-05-15 00:05:02,518] sync=False
[2018-05-15 00:05:02,519] Schedule=continuous_train_and_eval. Assuming that training is running on a single machine.
INFO:tensorflow:datashard_devices: ['gpu:0']
[2018-05-15 00:05:02,520] datashard_devices: ['gpu:0']
INFO:tensorflow:caching_devices: None
[2018-05-15 00:05:02,521] caching_device

/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
INFO:tensorflow:Done calling model_fn.
[2018-05-15 00:05:41,618] Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
[2018-05-15 00:05:41,621] Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
[2018-05-15 00:05:48,973] Graph was finalized.
INFO:tensorflow:Running local_init_op.
[2018-05-15 00:05:51,334] Running local_init_op.
INFO:tensorflow:Done running local_init_op.
[2018-05-15 00:05:51,500] Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /home/jdroo/t2t_train/observation_prediction_dices/transformer-transformer_small/model.ckpt.
[2018-05-15 00:06:21,379] Saving checkpoints for 1 into /home/jdroo/t2t_train/observation_prediction_dices/transformer-trans

In [ ]:
# See the transductions
# For each observation the top 3 predictions are shown with their respective log probability

! pygmentize -g sample_dices.observation
print("->-")
! pygmentize -g sample_dices.prediction